# Method Of Manufactured Solutions for SWIRL

In [1]:
# Set Up Libraries
%run 1-Set-Up.ipynb

In [2]:
# %load SourceSubstitution.py
# SourceSubstitution will substitute expressions for the following variables
def SourceSubstitution(S, \
                       A_analytic         , \
                       M_t_analytic       , \
                       M_x_analytical     , \
                       v_r_analytical     , \
                       v_t_analytical     , \
                       v_x_analytical     , \
                       p_analytical       , \
                       dp_dr_analytical   , \
                       dv_r_dr_analytical , \
                       dM_x_dr_analytical , \
                       dM_t_dr_analytical , \
                      ): 
    
    print('Input Source: ', S)
        S = S.subs({ \
                A:A_analytic       , \
                M_t:M_t_analytic   , \
                M_x:M_x_analytical , \
                v_r:v_r_analytical , \
                v_t:v_t_analytical , \
                v_x:v_x_analytical , \
                p:p_analytical     , \
                dp_dr:dp_dr_analytical, \
                dv_r_dr:dv_r_dr_analytical, \
                dM_x_dr:dM_x_dr_analytical, \
                dM_t_dr:dM_t_dr_analytical, \
               })
    print('Output Source', S)
    return S

In [3]:
# %load SourceTermVariables.py
# Defining symbolic variables needed for this code.
# all units are dimensionless! 

# radius, maximum radius, and ratio of specific heats
r, r_max, kappa = symbols('r r_max kappa')

# arbitrary constants
k = symbols('k', cls=IndexedBase)
one, two, three = symbols('one two three')
C = symbols('C')
x = symbols('x')

# reduced frequency
ak = symbols('ak')

# imaginary #, Speed of sound, azimuthal mode number, axial wavenumber 
i, A, m, gamma   = symbols('i A m gamma')
v_r, v_t, v_x, p = symbols('v_r v_t v_x p ')
M_t, M_x         = symbols('M_t M_x')
dp_dr, dv_r_dr   = symbols('dp_dr dv_r_dr')
dM_x_dr, dM_t_dr = symbols('dM_x_dr dM_t_dr')

#locations of inflection Points for tanh?
r2 , r3  = symbols('r2 r3')
j        = symbols('j')

### Theory
#### Speed of Sound Integration from Mean Flow

The goal was to use a function that would be harder to resolve with second
order schemes. A FORTRAN code was written from this notebook to create a              
summation of  $\tanh$  functions. The issue is implementing this with the       
current MMS workflow in SWIRL because of the symbolic expressions that would be needed for the source terms in the  Ax-λ Bx = S. For now we explicitly define 3 kink-anti-kink functions                                    


\begin{align*}                                                                                           
    \bar{A} = \sum_{j=1}^n R_{ij} + \sum_{j=1}^n L_{ij}                                                  
\end{align*}                                                                                             
where,                                                                                                   
\begin{align*}
    R_{ij} = A_j \tanh(B_j (x_i - x_j)) \\
    L_{ij} = A_j \tanh(B_j (x_j - x_n))           
\end{align*}
Letting $n = 3$
\begin{align*}
    \bar{A} &= S_{vert} + \sum_{j=1}^3 R_{ij} + \sum_{j=1}^3 L_{ij} \\        
    \bar{A} &=
    A_1 \tanh(B_1 (x_i - x_1))  +
    A_{2} \tanh(B_{2} (x_i - x_{2}))  +      
    A_{3} \tanh(B_{3} (x_i - x_3)) +  \\
    A_1 \tanh(B_1 (x_1 - x_n)) &+
    A_{2} \tanh(B_{2} (x_2 - x_{n}))  +
    A_{3} \tanh(B_{3} (x_3 - x_n))
\end{align*}

and,

\begin{align*}
   A_1 = A_2 = A_3 = k_1 \\
   B_1 = B_2 = B_3 = k_2
\end{align*}                                                                                                                                                                                             
                      

In [4]:
RightKink = \
k[1]*sp.tanh(k[2]*(r - r_max)) + \
k[1]*sp.tanh(k[2]*(r - r2)) + \
k[1]*sp.tanh(k[2]*(r - r3))

In [5]:
LeftKink = \
k[1]*sp.tanh(k[2]*(r_max - r_max)) + \
k[1]*sp.tanh(k[2]*(r2    - r_max)) + \
k[1]*sp.tanh(k[2]*(r3    - r_max))

#RightKink = Sum(sp.tanh(r - Indexed('r_loc',j)),(j,0,2))
#LeftKink = Sum(sp.tanh(Indexed('r_loc',j) - r_max),(j,0,2))
# had a hard time converting this to fortran! >:[

A_analytic        =  one + LeftKink + RightKink 
dA_analytic_dr    = diff(A_analytic,r)
dA_analytic_sq_dr = diff(A_analytic**two,r)
M_t_analytic      = sp.sqrt( \
                       r/((kappa-one)*A_analytic**two) * \
                       (dA_analytic_sq_dr))

### Defining perturbation variables

In [6]:
M_x_analytical = k[3]*sp.cos(k[3]*(r - r_max))
v_r_analytical = k[4]*sp.cos(k[4]*(r - r_max))
v_t_analytical = k[5]*sp.cos(k[5]*(r - r_max))
v_x_analytical = k[6]*sp.cos(k[6]*(r - r_max))
p_analytical   = k[7]*sp.cos(k[7]*(r - r_max))

dp_dr_analytical   = p_analytical.diff(r)
dv_r_dr_analytical = v_r_analytical.diff(r) 
dM_x_dr_analytical = M_x_analytical.diff(r)
dM_t_dr_analytical = M_t_analytic.diff(r)

# now we split source terms based on where they are in the A and B matricies

### Defining Source Terms

Initially the source terms were defined without mention of the indicies of the matricies they make up. In other words, there was no fore sight on the fact that these source terms are sums 
of the elements within A,B, and X. To investigate the source terms in greater detail, the FORTRAN code that calls the source terms will output the terms within the source term and then sum them, instead of just their sum.

#### Eigenvalue Decomposition
The SWIRL code performs an eigenvalue decomposition of this set of equations:

$$ [A]{x} = \lambda [B] {x} $$

which can be rearranged as,

$$ [A]{x} - \lambda [B] {x} = 0$$

Here, $x$ is an eigenvector composed of the perturbation variables, $v_r,v_{\theta},v_x,p$ and $\lambda$ is the associated eigenvalue, (Note: $\lambda = -i \bar{\gamma}$)

Writing this out we obtain ....

Linear System of Equations:
\begin{equation}
-
i \left(
\frac{k}{A} - \frac{m}{r} M_{\theta}
\right)
v_r 
-
\frac{2}{r} M_{\theta} v_{\theta} 
+
\frac{dp}{dr} 
+
\frac{(\kappa - 1)}{r} M_{\theta}^2 p
-
\lambda M_x v_r =S_1
\end{equation}

Using matrix notation,

\begin{equation}
A_{11}
x_1 
-
A_{12} x_2 
+
A_{14} x_4
-
\lambda B_{11} x_1 = S_1
\end{equation}

But $A_{14}$ and $A_{41}$ in Kousen's paper only has the derivative operator. Since I am currntly writing the matrix out term by term and not doing the matrix math to obtain the symbolic expressions, I will define $A_{14}$ with $dp/dr$ and $A_{41}$ with $dv_r/dr$
Similarly,
\begin{align}
A_{21} x_1 &-
A_{22} x_2 +
A_{24} x_4 &-
\lambda B_{22} x_2 &= S_2 \\
A_{31} x_1 &-
A_{33} x_3 &-
\lambda (B_{33} x_3 + B_{34} x_4) &= S_3\\
A_{41} x_1 &+
A_{42} x_2 +
A_{44} x_4 &- 
\lambda (B_{33} x_3 + B_{44} x_4) &= S_4
\end{align}
Now we can begin looking at the source terms, term by term. They each should also converge at a known rate


In [7]:
S = list(range(4)) 
S[0] = \
-i*(ak/A - (m/r)*M_t - gamma*M_x)*v_r  -\
\
(two/r)*M_t*v_t + dp_dr + ((kappa - one)/r)*(M_t**two)*p

S[1] = \
-i*(ak/A - (m/r)*M_t - gamma*M_x)*v_t + \
\
(M_t/r + dM_t_dr + ((kappa - one)/(two*r))*M_t**three)*v_r + i*m*p/r

S[2] = \
-i*(ak/A - (m/r)*M_t - gamma*M_x)*v_x + \
\
(dM_x_dr + ((kappa - one)/(two*r))*M_x*M_t**two)*v_r + i*one*gamma*p
 
S[3] = \
-i*(ak/A - (m/r)*M_t - gamma*M_x)*p   \
\
+ dv_r_dr + (((kappa + one)/(two*r))*M_t**two + one/r)*v_r + i*m*v_t/r + i*one*gamma*v_x

# Lets look at the source terms
# pprint('The linearized (unsteady) Euler Equations used in SWIRL:')
# pprint(('S_1=',S_1))
# pprint(('S_2=',S_2))
# pprint(('S_3=',S_3))
# pprint(('S_4=',S_4))

Now lets get these source terms by using the matrix form of the equations and see if they're
equal

In [ ]:
AA = sp.Matrix(sp.zeros(4,4))

AA[0,0] = -i*(ak/A - (m/r)*M_t)
AA[1,1] = AA[0,0]
AA[2,2] = AA[0,0]
AA[3,3] = AA[0,0]

AA[0,2] = 0
AA[1,2] = 0
AA[3,2] = 0
AA[2,1] = 0
AA[2,3] = 0

AA[1,3] = (i*m)/r
AA[3,1] = AA[1,3]

A12 = (-two/r)*M_t
A21 = M_t/r + dM_t_dr + (kappa - one)/(two*r)*M_t**three
A31 = dM_x_dr + (kappa - one)/(two*r)*M_t**two*M_x

AA[0,1] = A12
AA[1,0] = A21
AA[2,0] = A31

# note that these terms only have derivative operators , See Eqn 2.52

A41 = (1/v_r)*dv_r_dr + one/r + (kappa + one)/(two*r)*M_t**two
A14 = (1/p)*dp_dr + (kappa - one)/r * M_t**two

AA[3,0] = A41
AA[0,3] = A14

BB = sp.zeros(4,4)

BB[0,0] = M_x
BB[1,1] = BB[0,0]
BB[2,2] = BB[0,0]
BB[3,3] = BB[0,0]

BB[2,3] = one
BB[3,2] = BB[2,3]

XX = sp.Matrix([\
               [v_r] , \
               [v_t] , \
               [v_x] , \
               [p] , \
               ])

Lambda = -i*gamma

SS = AA*XX - Lambda*BB*XX

# Checking if the matrix expressions equal the linear system of equations
print(S[0].equals(SS[0]))
print(S[1].equals(SS[1]))
print(S[2].equals(SS[2]))
print(S[3].equals(SS[3]))

In [45]:
#now that we set the matricies up lets get our individual terms 

S_A = sp.Matrix(sp.zeros(4,4))
for j in range(len(S)):
    for i in range(len(S)):
        S_A[i,j] = [AA[i,j]*XX[j]]
        #print(S_A[i,j])

S_A11 = AA[0,0]*XX[0]
S_A12 = AA[0,1]*XX[1]
S_A13 = AA[0,2]*XX[2]
S_A14 = AA[0,3]*XX[3]

S_B11 = Lambda*BB[0,0]*XX[0]
S_B12 = Lambda*BB[0,1]*XX[1]
S_B13 = Lambda*BB[0,2]*XX[2]
S_B14 = Lambda*BB[0,3]*XX[3]
print(S_A[0,0].equals(S_A11))

True


In [8]:
S_A21 = AA[1,0]*XX[0]
S_A22 = AA[1,1]*XX[1]
S_A23 = AA[1,2]*XX[2]
S_A24 = AA[1,3]*XX[3]

S_B21 = Lambda*BB[1,0]*XX[0]
S_B22 = Lambda*BB[1,1]*XX[1]
S_B23 = Lambda*BB[1,2]*XX[2]
S_B24 = Lambda*BB[1,3]*XX[3]

S_A31 = AA[2,0]*XX[0]
S_A32 = AA[2,1]*XX[1]
S_A33 = AA[2,2]*XX[2]
S_A34 = AA[2,3]*XX[3]

S_B31 = Lambda*BB[2,0]*XX[0]
S_B32 = Lambda*BB[2,1]*XX[1]
S_B33 = Lambda*BB[2,2]*XX[2]
S_B34 = Lambda*BB[2,3]*XX[3]

S_A41 = AA[3,0]*XX[0]
S_A42 = AA[3,1]*XX[1]
S_A43 = AA[3,2]*XX[2]
S_A44 = AA[3,3]*XX[3]

S_B41 = Lambda*BB[3,0]*XX[0]
S_B42 = Lambda*BB[3,1]*XX[1]
S_B43 = Lambda*BB[3,2]*XX[2]
S_B44 = Lambda*BB[3,3]*XX[3]

print(S[0].equals(S_A11 + S_A12 + S_A13 + S_A14 -\
                (S_B11 + S_B12 + S_B13 + S_B14)))
    
print(S[1].equals(S_A21 + S_A22 + S_A23 + S_A24 -\
                (S_B21 + S_B22 + S_B23 + S_B24)))

print(S[2].equals(S_A31 + S_A32 + S_A33 + S_A34 -\
                (S_B31 + S_B32 + S_B33 + S_B34)))

print(S[3].equals(S_A41 + S_A42 + S_A43 + S_A44 -\
                (S_B41 + S_B42 + S_B43 + S_B44)))

#print(S_2.equals(S_A22+S_A22+S_A23+S_A24 - S_B22))
#print(S_3.equals(S_A31+S_A32+S_A33+S_A34 - S_B11))
#print(S_4.equals(S_A41+S_A42+S_A43+S_A44 - S_B11))
#print(SS[0]) 
# multiplying (1/p) and not (one/p) by dp_dr made  S_1 and SS[1] the same

#print(S_2)
#print(SS[1])
#print(S_3)
#print(SS[2]) # multiplied i gamma p term by "one" in S_3

#pprint(S_4)
#pprint(SS[3]) # multiplied by 1/v_r by d_v_dr and i v_x gamma terM by one

True
True
True
True
True
True
True
True


In [23]:
 
S_A_list = [S_A11]
for i in range(len(S)):
    S[i] = SourceSubstitution(S[i], \
                       A_analytic         , \
                       M_t_analytic   , \
                       M_x_analytical     , \
                       v_r_analytical     , \
                       v_t_analytical     , \
                       v_x_analytical     , \
                       p_analytical       , \
                       dp_dr_analytical   , \
                       dv_r_dr_analytical , \
                       dM_x_dr_analytical , \
                       dM_t_dr_analytical , \
                       )
    S_A_list[i] = SourceSubstitution(S[i], \
                       A_analytic         , \
                       M_t_analytic   , \
                       M_x_analytical     , \
                       v_r_analytical     , \
                       v_t_analytical     , \
                       v_x_analytical     , \
                       p_analytical       , \
                       dp_dr_analytical   , \
                       dv_r_dr_analytical , \
                       dM_x_dr_analytical , \
                       dM_t_dr_analytical , \
                       )
    

Input Source:  -i*(ak/(one + tanh((r - r2)*k[2])*k[1] + tanh((r - r3)*k[2])*k[1] + tanh((r - r_max)*k[2])*k[1] + tanh((r2 - r_max)*k[2])*k[1] + tanh((r3 - r_max)*k[2])*k[1]) - gamma*cos((r - r_max)*k[3])*k[3] - m*sqrt(r*two*((1 - tanh((r - r2)*k[2])**2)*k[1]*k[2] + (1 - tanh((r - r3)*k[2])**2)*k[1]*k[2] + (1 - tanh((r - r_max)*k[2])**2)*k[1]*k[2])/((kappa - one)*(one + tanh((r - r2)*k[2])*k[1] + tanh((r - r3)*k[2])*k[1] + tanh((r - r_max)*k[2])*k[1] + tanh((r2 - r_max)*k[2])*k[1] + tanh((r3 - r_max)*k[2])*k[1])))/r)*cos((r - r_max)*k[4])*k[4] - sin((r - r_max)*k[7])*k[7]**2 - two*sqrt(r*two*((1 - tanh((r - r2)*k[2])**2)*k[1]*k[2] + (1 - tanh((r - r3)*k[2])**2)*k[1]*k[2] + (1 - tanh((r - r_max)*k[2])**2)*k[1]*k[2])/((kappa - one)*(one + tanh((r - r2)*k[2])*k[1] + tanh((r - r3)*k[2])*k[1] + tanh((r - r_max)*k[2])*k[1] + tanh((r2 - r_max)*k[2])*k[1] + tanh((r3 - r_max)*k[2])*k[1])))*cos((r - r_max)*k[5])*k[5]/r + (r*two*((1 - tanh((r - r2)*k[2])**2)*k[1]*k[2] + (1 - tanh((r - r3)*k[2])**2

Output Source i*m*cos((r - r_max)*k[7])*k[7]/r - i*(ak/(one + tanh((r - r2)*k[2])*k[1] + tanh((r - r3)*k[2])*k[1] + tanh((r - r_max)*k[2])*k[1] + tanh((r2 - r_max)*k[2])*k[1] + tanh((r3 - r_max)*k[2])*k[1]) - gamma*cos((r - r_max)*k[3])*k[3] - m*sqrt(r*two*((1 - tanh((r - r2)*k[2])**2)*k[1]*k[2] + (1 - tanh((r - r3)*k[2])**2)*k[1]*k[2] + (1 - tanh((r - r_max)*k[2])**2)*k[1]*k[2])/((kappa - one)*(one + tanh((r - r2)*k[2])*k[1] + tanh((r - r3)*k[2])*k[1] + tanh((r - r_max)*k[2])*k[1] + tanh((r2 - r_max)*k[2])*k[1] + tanh((r3 - r_max)*k[2])*k[1])))/r)*cos((r - r_max)*k[5])*k[5] + (sqrt(r*two*((1 - tanh((r - r2)*k[2])**2)*k[1]*k[2] + (1 - tanh((r - r3)*k[2])**2)*k[1]*k[2] + (1 - tanh((r - r_max)*k[2])**2)*k[1]*k[2])/((kappa - one)*(one + tanh((r - r2)*k[2])*k[1] + tanh((r - r3)*k[2])*k[1] + tanh((r - r_max)*k[2])*k[1] + tanh((r2 - r_max)*k[2])*k[1] + tanh((r3 - r_max)*k[2])*k[1])))/r + sqrt(r*two*((1 - tanh((r - r2)*k[2])**2)*k[1]*k[2] + (1 - tanh((r - r3)*k[2])**2)*k[1]*k[2] + (1 - tanh((

IndexError: list assignment index out of range

In [ ]:
SSS_1 = S_A11 + S_A12 + S_A13 + S_A14 -\
       S_B11 + S_B12 + S_B13 + S_B14
pprint(SSS_1)

In [ ]:
%run create-fortran-file-sound-speed.ipynb
%run create-fortran-file-LEE-source-terms.ipynb
%run create-fortran-file-perturbation-variables.ipynb
# need to include axial mach number in sound speed fortran routine
# need to break up source terms into its components